# Análisis de negociación de deudas en mora prolongada (>180 días)

**Prueba técnica – Analista de Datos**  
**Autor:** Santiago García  
**Fecha:** 22/01/2026  

**Objetivo:** identificar factores asociados a la probabilidad de llegar a acuerdos de pago en cartera castigada.

## 1. Carga de datos

In [1]:
import pandas as pd

df = pd.read_csv("../data/prueba_analist_de_datos_crecere.csv")

df.head()

,antiguedad_deuda,deuda_total,mujer,edad,region_bogota,propuesta_cuotas,propuesta_descuento,estrategia_agresiva,origen_deudor,estado_civil,tiene_propiedades,producto_origen_deuda,beneficiario_subsidios,helado_favorito,mes,ingreso,con_acuerdo
0,491,4.819506e+07,0,42.225370,0,5,0.062792,1,Colombia,union libre,0,credito vehiculo,0,ron con pasas,5,4.244878e+06,0
1,615,6.192111e+07,0,27.134840,0,6,0.326825,0,Colombia,soltero/a,0,otro,0,chocolate,7,2.860102e+06,0
2,850,3.505055e+07,0,42.595440,0,6,0.579158,0,Colombia,union libre,0,credito vehiculo,0,otro,2,4.547806e+06,0
3,1243,8.598943e+07,0,27.001417,1,9,0.484808,0,Venzuela,casado/a,0,otro,0,ron con pasas,5,4.917511e+06,0
4,416,1.144557e+07,0,39.581093,0,9,0.259015,0,Colombia,soltero/a,1,tarjeta credito,0,otro,12,4.045892e+06,0


## 2. Exploración inicial del dataset

In [2]:
df.shape

(1000, 17)

In [3]:
df.columns

Index(['antiguedad_deuda', 'deuda_total', 'mujer', 'edad', 'region_bogota',
       'propuesta_cuotas', 'propuesta_descuento', 'estrategia_agresiva',
       'origen_deudor', 'estado_civil', 'tiene_propiedades',
       'producto_origen_deuda', 'beneficiario_subsidios', 'helado_favorito',
       'mes', 'ingreso', 'con_acuerdo'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   antiguedad_deuda        1000 non-null   int64  
 1   deuda_total             1000 non-null   float64
 2   mujer                   1000 non-null   int64  
 3   edad                    1000 non-null   float64
 4   region_bogota           1000 non-null   int64  
 5   propuesta_cuotas        1000 non-null   int64  
 6   propuesta_descuento     1000 non-null   float64
 7   estrategia_agresiva     1000 non-null   int64  
 8   origen_deudor           1000 non-null   object 
 9   estado_civil            1000 non-null   object 
 10  tiene_propiedades       1000 non-null   int64  
 11  producto_origen_deuda   1000 non-null   object 
 12  beneficiario_subsidios  1000 non-null   int64  
 13  helado_favorito         1000 non-null   object 
 14  mes                     1000 non-null   i

In [5]:
df.describe()

,antiguedad_deuda,deuda_total,mujer,edad,region_bogota,propuesta_cuotas,propuesta_descuento,estrategia_agresiva,tiene_propiedades,beneficiario_subsidios,mes,ingreso,con_acuerdo
count,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000
mean,764.654000,4.458539e+07,0.508000,35.016629,0.305000,6.49700,0.293398,0.495000,0.164000,0.084000,6.477000,3.685861e+06,0.088000
std,337.422341,2.640343e+07,0.500186,8.720312,0.460638,3.48628,0.173220,0.500225,0.370461,0.277527,3.559233,1.183488e+06,0.283437
min,182.000000,9.539292e+05,0.000000,18.000000,0.000000,1.00000,0.000064,0.000000,0.000000,0.000000,1.000000,6.904236e+05,0.000000
25%,482.000000,2.123832e+07,0.000000,29.314159,0.000000,3.00000,0.147190,0.000000,0.000000,0.000000,3.000000,2.807265e+06,0.000000
50%,745.500000,4.375615e+07,1.000000,34.759870,0.000000,7.00000,0.293081,0.000000,0.000000,0.000000,7.000000,3.667244e+06,0.000000
75%,1054.250000,6.835439e+07,1.000000,41.319729,1.000000,9.00000,0.439206,1.000000,0.000000,0.000000,10.000000,4.495536e+06,0.000000
max,1350.000000,8.989998e+07,1.000000,61.808757,1.000000,12.00000,0.598384,1.000000,1.000000,1.000000,12.000000,6.996183e+06,1.000000


## 3. Variable objetivo: acuerdo de pago

In [6]:
df["con_acuerdo"].value_counts()

con_acuerdo
0    912
1     88
Name: count, dtype: int64

In [7]:
df["con_acuerdo"].mean()

np.float64(0.088)

## 4. Análisis por variables explicativas

In [8]:
df.groupby(
    pd.cut(df["propuesta_descuento"], bins=5),
    observed=False
)["con_acuerdo"].mean()

propuesta_descuento
(-0.000534, 0.12]    0.062500
(0.12, 0.239]        0.072464
(0.239, 0.359]       0.069652
(0.359, 0.479]       0.095238
(0.479, 0.598]       0.143590
Name: con_acuerdo, dtype: float64

In [9]:
df.groupby("estrategia_agresiva")["con_acuerdo"].mean()

estrategia_agresiva
0    0.021782
1    0.155556
Name: con_acuerdo, dtype: float64

In [10]:
df.groupby(
    pd.cut(df["ingreso"], bins=5),
    observed=False
)["con_acuerdo"].mean()

ingreso
(684117.854, 1951575.544]     0.044776
(1951575.544, 3212727.474]    0.063973
(3212727.474, 4473879.404]    0.099476
(4473879.404, 5735031.334]    0.100503
(5735031.334, 6996183.263]    0.145455
Name: con_acuerdo, dtype: float64

In [11]:
df.groupby("region_bogota")["con_acuerdo"].mean()

region_bogota
0    0.077698
1    0.111475
Name: con_acuerdo, dtype: float64

In [12]:
df.groupby("region_bogota")["ingreso"].mean()

region_bogota
0    3.205967e+06
1    4.779389e+06
Name: ingreso, dtype: float64

## 5. Análisis multivariado: factores asociados al acuerdo de pago

In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 

X = df[["propuesta_descuento", "ingreso", "estrategia_agresiva", "region_bogota"]]
y = df["con_acuerdo"]

X = sm.add_constant(X)

model = sm.Logit(y, X).fit()

print(model.summary())

print("\n--- Odds Ratios (Impacto relativo) ---")
print(np.exp(model.params))

Optimization terminated successfully.
         Current function value: 0.257757
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:            con_acuerdo   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      995
Method:                           MLE   Df Model:                            4
Date:                Thu, 22 Jan 2026   Pseudo R-squ.:                  0.1347
Time:                        21:11:20   Log-Likelihood:                -257.76
converged:                       True   LL-Null:                       -297.89
Covariance Type:            nonrobust   LLR p-value:                 1.535e-16
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -5.7267      0.608     -9.425      0.000      -6.918      -4.536
pr

## 6. Exportación de resultados para visualización

In [14]:
tabla_descuento = df.groupby(
    pd.cut(df["propuesta_descuento"], bins=5),
    observed=False
)["con_acuerdo"].mean()

tabla_descuento.to_csv(
    "../outputs/tabla_descuento.csv",
    sep=";",
    decimal=",",
    float_format="%.6f"
)


tabla_region = df.groupby("region_bogota")["con_acuerdo"].mean()

tabla_region.to_csv(
    "../outputs/tabla_region.csv",
    sep=";",
    decimal=",",
    float_format="%.6f"
)


tabla_ingreso = df.groupby(
    pd.cut(df["ingreso"], bins=5),
    observed=False
)["con_acuerdo"].mean()

tabla_ingreso.to_csv(
    "../outputs/tabla_ingreso.csv",
    sep=";",
    decimal=",",
    float_format="%.6f"
)


tabla_estrategia = df.groupby("estrategia_agresiva")["con_acuerdo"].mean()

tabla_estrategia.to_csv(
    "../outputs/tabla_estrategia.csv",
    sep=";",
    decimal=",",
    float_format="%.6f"
)